In [2]:
from sklearn.datasets import load_files
import pandas as pd
import os

In [25]:
PATH_TO_DATA = 'data/imdb/'
RANDOM_STATE = 17

In [3]:
data = pd.read_csv(os.path.join(PATH_TO_DATA, 'train.csv'))
data['sentiment'] = data['sentiment'].apply(lambda x: 1 if x=="positive" else 0)
train_reviews = data.iloc[:data.shape[0]//2]
test_reviews = data.iloc[data.shape[0]//2:]


In [4]:
train_reviews.shape, test_reviews.shape

((25000, 2), (25000, 2))

BOW - мешок слов

In [5]:
import numpy as np
from scipy.sparse import csr_matrix

In [6]:
a = np.zeros((5,5))
a[0, 3] = 1
a[4, 4] = 6
a[2, 2] = 5
a[3, 1] = 4
a[3, 2] = 4
a[1, 1] = 7

In [7]:
a

array([[0., 0., 0., 1., 0.],
       [0., 7., 0., 0., 0.],
       [0., 0., 5., 0., 0.],
       [0., 4., 4., 0., 0.],
       [0., 0., 0., 0., 6.]])

In [8]:
pd.DataFrame(a)

,0,1,2,3,4
0,0.0,0.0,0.0,1.0,0.0
1,0.0,7.0,0.0,0.0,0.0
2,0.0,0.0,5.0,0.0,0.0
3,0.0,4.0,4.0,0.0,0.0
4,0.0,0.0,0.0,0.0,6.0


In [9]:
b = csr_matrix(a)
b

<5x5 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [10]:
b.todense()

matrix([[0., 0., 0., 1., 0.],
        [0., 7., 0., 0., 0.],
        [0., 0., 5., 0., 0.],
        [0., 4., 4., 0., 0.],
        [0., 0., 0., 0., 6.]])

In [11]:
b.nonzero()

(array([0, 1, 2, 3, 3, 4]), array([3, 1, 2, 1, 2, 4]))

In [12]:
b.data

array([1., 7., 5., 4., 4., 6.])

In [13]:
train_reviews.iloc[:, 0]

0        One of the other reviewers has mentioned that ...
1        A wonderful little production. <br /><br />The...
2        I thought this was a wonderful way to spend ti...
3        Basically there's a family where a little boy ...
4        Petter Mattei's "Love in the Time of Money" is...
                               ...                        
24995    This movie was a real torture fest to sit thro...
24996    John Wayne & Albert Dekker compete for oil rig...
24997    Tarantino once remarked on a melodrama from th...
24998    Aah yes the workout show was a great. Not only...
24999    This film should have never been made. Honestl...
Name: review, Length: 25000, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
cv = CountVectorizer(ngram_range=(1, 2))
X_train_sparse = cv.fit_transform(train_reviews.iloc[:, 0])
X_test_sparse = cv.transform(test_reviews.iloc[:, 0])


In [57]:
X_train_sparse.shape, X_test_sparse.shape


((25000, 1513369), (25000, 1513369))

In [51]:
y_train, y_test = train_reviews.iloc[:, 1], test_reviews.iloc[:, 1]

In [52]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score

In [53]:
logit = LogisticRegression(random_state=RANDOM_STATE, n_jobs=-1, solver='lbfgs')
sgd_logit = SGDClassifier(max_iter=int(10**6/ X_test_sparse.shape[0]), random_state=RANDOM_STATE, n_jobs=-1)

In [54]:
%%time

logit.fit(X_train_sparse, y_train)

CPU times: total: 516 ms
Wall time: 30.6 s


LogisticRegression(n_jobs=-1, random_state=17)

In [55]:
%%time

sgd_logit.fit(X_train_sparse, y_train)

CPU times: total: 2.23 s
Wall time: 1.42 s


SGDClassifier(max_iter=40, n_jobs=-1, random_state=17)

In [1]:
accuracy_score(y_test, logit.predict(X_test_sparse)), accuracy_score(y_test, sgd_logit.predict(X_test_sparse)), 

NameError: name 'accuracy_score' is not defined